<a href="https://colab.research.google.com/github/tpmai-nguyen/K224131544_NguyenThiPhuongMai_Model/blob/main/Copy_of_K224131544_NguyenThiPhuongMai_TO52.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Nhóm 7**

1.   Nguyễn Thị Phương Mai K224131544
2.   Nguyễn Hồ Minh Nguyệt K224131549
3.   Phạm Thị Hoài Thư K224131560


# **PHÂN TÍCH MÔ TẢ**

Thu thập và khám phá dữ liệu

In [ ]:
import pandas as pd
import requests
from io import StringIO

url='https://drive.google.com/file/d/1viztnEQaMsATM7efkgprx87ybFXHNQly/view?usp=drive_link'

file_id = url.split('/')[-2]
dwn_url='https://drive.google.com/uc?export=download&id=' + file_id
url2 = requests.get(dwn_url).text
csv_raw = StringIO(url2)
df = pd.read_csv(csv_raw)
print(df.head())

In [ ]:
df.info()

In [ ]:
display(df.describe())
df.describe(include=['O'])

In [ ]:
df.isnull().sum()

Tiền xử lý dữ liệu

In [ ]:
for column in df.columns:
    if df[column].isnull().any():
        if df[column].dtype == 'object':
            df[column].fillna(df[column].mode()[0], inplace=True)
        else:
            df[column].fillna(df[column].mean(), inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df.columns = [col.lower().replace(' ', '_') for col in df.columns]
df.rename(columns=lambda x: x.replace("(", "").replace(")", ""), inplace=True)
df.info()

In [ ]:
df.drop(["customer_email","customer_fname", "customer_lname","customer_id","customer_password","customer_street","customer_zipcode",
         "department_id","department_name","order_item_cardprod_id","order_state", "product_description", "product_status","product_image"]
        ,axis= 1 , inplace= True)

In [ ]:
df.duplicated().sum()

In [ ]:
df[df.apply(lambda row: row.astype(str).str.contains('\?').any(),axis=1)]

In [ ]:
for i in df.columns:
    for j in range(len(df[i])):
        value = str(df[i][j])
        if '?' in value:
            df.at[j, i] = value.replace('?','')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
df[["days_for_shipping_real"]].hist(bins=50, ax=axes[0]);
df[["days_for_shipping_real"]].boxplot(ax=axes[1], vert=False);

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from typing import Tuple
from sklearn.base import BaseEstimator, TransformerMixin
def find_boxplot_boundaries(
    col: pd.Series, whisker_coeff: float = 1.5) -> Tuple[float, float]:
    Q1 = col.quantile(0.25)
    Q3 = col.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - whisker_coeff * IQR
    upper = Q3 + whisker_coeff * IQR
    return lower, upper

class BoxplotOutlierClipper(BaseEstimator, TransformerMixin):
    def __init__(self, whisker_coeff: float = 1.5):
        self.whisker = whisker_coeff
        self.lower = None
        self.upper = None

    def fit(self, X: pd.Series):
        self.lower, self.upper = find_boxplot_boundaries(X, self.whisker)
        return self

    def transform(self, X):
        return X.clip(self.lower, self.upper)

clipped_benefit_per_order = BoxplotOutlierClipper().fit_transform(df["days_for_shipping_real"])

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
clipped_benefit_per_order.hist(bins=50, ax=axes[0])
clipped_benefit_per_order.to_frame().boxplot(ax=axes[1], vert=False);

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
df[["days_for_shipment_scheduled"]].hist(bins=50, ax=axes[0]);
df[["days_for_shipment_scheduled"]].boxplot(ax=axes[1], vert=False);

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from typing import Tuple
from sklearn.base import BaseEstimator, TransformerMixin
def find_boxplot_boundaries(
    col: pd.Series, whisker_coeff: float = 1.5) -> Tuple[float, float]:
    Q1 = col.quantile(0.25)
    Q3 = col.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - whisker_coeff * IQR
    upper = Q3 + whisker_coeff * IQR
    return lower, upper

class BoxplotOutlierClipper(BaseEstimator, TransformerMixin):
    def __init__(self, whisker_coeff: float = 1.5):
        self.whisker = whisker_coeff
        self.lower = None
        self.upper = None

    def fit(self, X: pd.Series):
        self.lower, self.upper = find_boxplot_boundaries(X, self.whisker)
        return self

    def transform(self, X):
        return X.clip(self.lower, self.upper)

clipped_order_profit_per_order = BoxplotOutlierClipper().fit_transform(df["days_for_shipment_scheduled"])

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
clipped_order_profit_per_order.hist(bins=50, ax=axes[0])
clipped_order_profit_per_order.to_frame().boxplot(ax=axes[1], vert=False);

In [ ]:
df_1=df.copy()
df_1

Trực quan hóa dữ liệu

In [ ]:
df['market'].value_counts().plot.pie(autopct='%1.1f%%', startangle=90, colors=sns.color_palette('pastel'))
plt.title('Market Distribution')
plt.ylabel('')
plt.show()

df['customer_segment'].value_counts().plot.pie(autopct='%1.1f%%', startangle=90, colors=sns.color_palette('pastel'))
plt.title('Customer Segment Distribution')
plt.ylabel('')
plt.show()


In [ ]:
category_counts = df['category_name'].value_counts()
plt.figure(figsize=(12, 6))
sns.barplot(x=category_counts.index, y=category_counts.values, palette='coolwarm')
plt.title('Count of Each Category Name')
plt.xlabel('Category Name')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.show()

product_counts = df['product_name'].value_counts().nlargest(10)
plt.figure(figsize=(12, 6))
sns.barplot(x=product_counts.index, y=product_counts.values, palette='coolwarm')
plt.title('Top 10 Most Common Products')
plt.xlabel('Product Name')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
sns.pairplot(df[['sales_per_customer', 'benefit_per_order', 'order_item_product_price']], diag_kind='kde', palette='coolwarm')
plt.suptitle('Pair Plot of Continuous Variables', y=1.02)
plt.show()

In [ ]:
from sklearn.preprocessing import LabelEncoder

object_columns = df.select_dtypes(include=['object']).columns.tolist()

for column in object_columns:
    df[column] = df[column].astype(str)
    label_encoder = LabelEncoder()
    df[column] = label_encoder.fit_transform(df[column])

df.info()

In [ ]:
df['late_days'] = df['days_for_shipping_real'] - df['days_for_shipment_scheduled']
df['late_days']

Trực quan hóa

In [ ]:
plt.figure(figsize=(14, 6))
sns.histplot(df['days_for_shipping_real'], kde=True, color='blue')
plt.title('Distribution of Days for Shipping (real)')
plt.xlabel('days_for_shipping_real')
plt.ylabel('Frequency')
plt.show()

plt.figure(figsize=(14, 6))
sns.histplot(df['days_for_shipment_scheduled'], kde=True, color='blue')
plt.title('Distribution of Days for shipment (scheduled)')
plt.xlabel('days_for_shipment_scheduled')
plt.ylabel('Frequency')
plt.show()

In [ ]:
numeric_df = df.select_dtypes(include=['number'])
numeric_df.corr()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(16,14))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap='coolwarm', vmin=-1, vmax=1, annot_kws={"size": 8}, fmt=".2f")
plt.title('Correlation Heatmap')

# **PHÂN TÍCH DỰ ĐOÁN**

Dự đoán rủi ro giao hàng trễ

In [ ]:
df_copy=df[['type',"days_for_shipment_scheduled","delivery_status","late_delivery_risk","category_id",
            "customer_city","customer_country","customer_segment","customer_state","latitude","longitude","order_country","order_city",
            "order_item_product_price","order_item_quantity","order_status","product_card_id","product_price",
            "shipping_date_dateorders","shipping_mode","late_days","order_date_dateorders","order_region","market"]]


In [ ]:
c=df.corr()
print(c['late_delivery_risk'])

In [ ]:
delivery_status_data = df_1['delivery_status']

delivery_status_summary = delivery_status_data.value_counts()

plt.figure(figsize=(10, 6))
sns.set(style="whitegrid")
sns.countplot(data=df_1, x='delivery_status', order=delivery_status_summary.index, palette='coolwarm')
plt.title('Delivery Status Distribution')
plt.xlabel('Delivery Status')
plt.ylabel('Count')

print("Delivery Status Summary:")
print(delivery_status_summary)

In [ ]:
df_copy['late_days'].value_counts().plot.pie(legend = ["0", "1"])

In [ ]:
delivery_status_data = df_1['delivery_status']
shipping_mode_data = df_1['shipping_mode']

cross_tab = pd.crosstab(shipping_mode_data, delivery_status_data)

cross_tab_percent = cross_tab.div(cross_tab.sum(1), axis=0) * 100

plt.figure(figsize=(12, 8))
sns.set(style="whitegrid")
cross_tab_percent.plot(kind="bar", stacked=True, colormap='Set2')
plt.title('Delivery Status by Shipping Mode (Percentage)')
plt.xlabel('Shipping Mode')
plt.ylabel('Percentage')

print("Cross-Tabulation (Contingency Table) with Percentages:")
print(cross_tab_percent)

In [ ]:
late_delivery_data = df_1[df_1['delivery_status'] == 'Late delivery']
late_by_product = late_delivery_data['category_name'].value_counts().nlargest(10).reset_index()

plt.figure(figsize=(12, 6))
sns.barplot(x=late_by_product.index, y=product_counts.values, palette='coolwarm')
plt.title('Top 10 Danh mục hàng giao hàng muộn')
plt.xlabel('Tên danh mục')
plt.ylabel('Số lượng')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
group = df_1.groupby(['market', 'delivery_status']).market.count().unstack()
group.plot(kind='bar')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier, BaggingClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_copy.info()

In [ ]:
target1 = 'late_delivery_risk'
if target1 not in df_1.columns:
    raise ValueError(f"The target variable '{target1}' is not present in the dataset.")

In [ ]:
X = df_copy.drop(target1, axis=1)
y = df_copy[target1]

from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)

X_balanced, y_balanced = smote.fit_resample(X, y)
y = y_balanced
X = X_balanced

class_counts = pd.Series(y_balanced).value_counts()
print("Số lượng lớp 0 sau khi cân bằng:", class_counts[0])
print("Số lượng lớp 1 sau khi cân bằng:", class_counts[1])


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

from sklearn.decomposition import PCA
from sklearn.model_selection import  KFold, cross_val_score, StratifiedKFold
import numpy as np
import pandas as pd

models = [
    ("Decision Tree", DecisionTreeClassifier()),
    ("Random Forest", RandomForestClassifier()),
    ("Gaussian Naive Bayes", GaussianNB()),
    ("Logistic Regression", LogisticRegression()),
    ("KNeighbors", KNeighborsClassifier())
]

output = {'Model': [], 'Accuracy': [], 'Precision': [], 'Recall': [], 'F1-score': []}
for model_name, model in models:
    model.fit(X_train, y_train)
    accuracy_scores = cross_val_score(model, X_test, y_test,  scoring='accuracy')
    precision_scores = cross_val_score(model, X_test, y_test,  scoring='precision')
    recall_scores = cross_val_score(model, X_test, y_test,   scoring='recall')
    f1_scores = cross_val_score(model, X_test, y_test,   scoring='f1')
    output['Model'].append(model_name)
    output['Accuracy'].append(np.mean(accuracy_scores))
    output['Precision'].append(np.mean(precision_scores))
    output['Recall'].append(np.mean(recall_scores))
    output['F1-score'].append(np.mean(f1_scores))

output_df = pd.DataFrame(output)
output_df

In [ ]:
from sklearn.decomposition import PCA
from sklearn.model_selection import  KFold, cross_val_score, StratifiedKFold
import numpy as np
import pandas as pd

pca = PCA(n_components=0.8)

X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

models = [
    ("Decision Tree", DecisionTreeClassifier()),
    ("Random Forest", RandomForestClassifier()),
    ("Gaussian Naive Bayes", GaussianNB()),
    ("Logistic Regression", LogisticRegression()),
    ("KNeighbors", KNeighborsClassifier())
]

output = {'Model': [], 'Accuracy': [], 'Precision': [], 'Recall': [], 'F1-score': [],}
for model_name, model in models:
    model.fit(X_train_pca, y_train)
    accuracy_scores = cross_val_score(model, X_test_pca,  y_test, scoring='accuracy')
    precision_scores = cross_val_score(model, X_test_pca,  y_test,  scoring='precision')
    recall_scores = cross_val_score(model, X_test_pca,  y_test,  scoring='recall')
    f1_scores = cross_val_score(model, X_test_pca,  y_test,  scoring='f1')
    output['Model'].append(model_name)
    output['Accuracy'].append(np.mean(accuracy_scores))
    output['Precision'].append(np.mean(precision_scores))
    output['Recall'].append(np.mean(recall_scores))
    output['F1-score'].append(np.mean(f1_scores))

output_df = pd.DataFrame(output)
output_df

In [ ]:
best_model_name = output_df.loc[output_df['Accuracy'].idxmax(), 'Model']
best_model_classifier = models[best_model_name]
print(f"Best Model: {best_model_name}")

with open('best_model.pkl', 'wb') as f:
    pickle.dump(best_model_classifier, f)
### COde sửa lại nha
from google.colab import files
# Change the filename to match the saved file name
files.download('best_model.pkl')

In [ ]:
# Test

# Function to perform frequency encoding
def frequency_encoding(df, column):
    frequency_map = df[column].value_counts(normalize=True)
    df[column + '_freq_encoded'] = df[column].map(frequency_map)
    return df

# New data to predict
instances_to_predict = pd.DataFrame({
    'Type': ['DEBIT','DEBIT'],
    'Days for shipment (scheduled)': [5,6],
    'Delivery Status': ['Advance shipping','Advance shipping'],
    'Category Id': [17,17],
    'Category Name': ['Cleats','Cleats'],
    'Customer City': ['Los Angeles','Los Angeles'],
    'Customer Country': ['EE. UU.', 'EE. UU.'],
    'Customer Segment': ['Corporate','Corporate'],
    'Customer State': ['NY','NY'],
    'Latitude': [17.24253835,17.24253835],
    'Longitude': [-65.03704823,-65.03704823],
    'Order City': ['Bikaner','Bikaner'],
    'Order Country': ['India','India'],
    'Order Item Product Price': [327.75,327.75],
    'Order Item Quantity': [2,2],
    'Order Status': ['COMPLETE','COMPLETE'],
    'Product Card Id': [1360,1360],
    'Product Price': [327.75,327.75],
    'Order Region': ['Southeast Asia','Southeast Asia'],
    'Market': ['Asia','Asia']
})

# Columns to be frequency encoded
columns_to_encode = ['Type', 'Delivery Status','Category Name','Customer City','Customer Country','Customer Segment',
                    'Customer State','Order City','Order Country','Order Status','Order Region','Market']

# Perform frequency encoding for each column
for column in columns_to_encode:
    instances_to_predict = frequency_encoding(instances_to_predict, column)

# Drop the original categorical columns
instances_to_predict.drop(columns=columns_to_encode, inplace=True)

# Standardize the features
scaler = StandardScaler()
instances_to_predict_scaled = scaler.fit_transform(instances_to_predict)

# Simulated training data and labels
# Replace these with your actual training data and labels
X_train = np.random.rand(10, instances_to_predict_scaled.shape[1])
y_train = np.random.randint(0, 2, 10)  # Random binary labels

# Train the model
gb_model = RandomForestClassifier()
gb_model.fit(X_train, y_train)

# Predict whether delivery is late or not
predictions = gb_model.predict(instances_to_predict_scaled)

# Print the predictions
print("Case 1: Predicted Late Delivery:", "Yes" if predictions[0] == 1 else "No")
print("Case 2: Predicted Late Delivery:", "Yes" if predictions[1] == 1 else "No")



Dự đoán doanh số mỗi khách hàng tạo ra

In [ ]:
df_copy2=df[["sales","delivery_status","category_id","customer_city","customer_country","customer_segment","customer_state","order_country","order_city",
            "order_item_product_price","order_item_quantity","order_status","product_card_id","product_price","order_region","market"]]

In [ ]:
c=df.corr()
print(c['sales'])

In [ ]:
df['order_date_dateorders'] = pd.to_datetime(df['order_date_dateorders'])

plt.figure(figsize=(14, 6))
df.groupby('order_date_dateorders')['sales'].sum().plot()
plt.title('Total Sales Over Time')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.show()

In [ ]:
df_1['order_date_dateorders)'] = pd.to_datetime(df_1['order_date_dateorders'], errors='coerce')
df_1['sales'] = pd.to_numeric(df_1['sales'], errors='coerce')
df_1_cleaned = df_1.dropna(subset=['sales', 'order_date_dateorders)'])
df_1_cleaned['Month'] = df_1_cleaned['order_date_dateorders)'].dt.to_period('M')
monthly_sales_total = df_1_cleaned.groupby('Month')['sales'].mean().reset_index()
monthly_sales_total['Month'] = monthly_sales_total['Month'].dt.to_timestamp()

plt.figure(figsize=(14, 6))
sns.lineplot(x=monthly_sales_total['Month'], y=monthly_sales_total['sales'], color='blue', linewidth=2)
plt.title('Sales Over Time (Monthly Average)')
plt.xlabel('Month')
plt.ylabel('Sales')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
market_sales = df_1.groupby('market')['sales'].sum()

top_n_markets = market_sales.sort_values(ascending=False).head(10)

plt.figure(figsize=(12, 6))
top_n_markets.plot(kind='bar')
plt.title('Top Markets by Total Sales')
plt.xlabel('Market')
plt.ylabel('Total Sales')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.barplot(x='category_name', y='sales', data=df_1)
plt.title('Doanh thu theo loại hàng')
plt.xlabel('Loại hàng')
plt.ylabel('Doanh thu')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
target2 = 'sales'
if target2 not in df_1.columns:
    raise ValueError(f"The target variable '{target2}' is not present in the dataset.")

In [ ]:
!pip install scikit-learn
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error, r2_score
from sklearn.pipeline import make_pipeline

X = df_copy2.drop(target2, axis=1)
y = df_copy2[target2]

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.3, random_state=42)

scaler = StandardScaler()
X2_train = scaler.fit_transform(X2_train)
X2_test = scaler.transform(X2_test)



models = {
    'Multiple Linear Regression': LinearRegression(),
    'Polynomial Regression': make_pipeline(PolynomialFeatures(degree=2), LinearRegression()),
    'Support Vector Regression': SVR(),
    'Decision Tree Regression': DecisionTreeRegressor(),
    'Random Forest Regression': RandomForestRegressor()
}

results = {}
for name, model in models.items():
    model.fit(X2_train, y2_train)
    y2_pred = model.predict(X2_test)

    mse = mean_squared_error(y2_test, y2_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y2_test, y2_pred)
    msle = mean_squared_log_error(y2_test, y2_pred)
    r_square = r2_score(y2_test, y2_pred)
    results[name] = {'MSE': mse, 'RMSE': rmse, 'MAE': mae, 'MSLE': msle, 'R^2': r_square}

for model_name, metrics in results.items():
    print(f"{model_name} performance:")
    for metric, value in metrics.items():
        print(f"  {metric}: {value:.4f}")
    print()

In [ ]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.3, random_state=42)

scaler = StandardScaler()
X2_train = scaler.fit_transform(X2_train)
X2_test = scaler.transform(X2_test)

models = {
    'Multiple Linear Regression': LinearRegression(),
    'Polynomial Regression': make_pipeline(PolynomialFeatures(degree=2), LinearRegression()),
    'Support Vector Regression': SVR(),
    'Decision Tree Regression': DecisionTreeRegressor(max_depth=10), # Limit tree depth
    'Random Forest Regression': RandomForestRegressor(n_estimators=100, max_depth=10) # More estimators, limit depth
}

results = {}
for name, model in models.items():
    model.fit(X2_train, y2_train)
    y2_pred = model.predict(X2_test)

    mse = mean_squared_error(y2_test, y2_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y2_test, y2_pred)
    msle = mean_squared_log_error(y2_test, y2_pred)
    r_square = r2_score(y2_test, y2_pred)
    results[name] = {'MSE': mse, 'RMSE': rmse, 'MAE': mae, 'MSLE': msle, 'R^2': r_square}

for model_name, metrics in results.items():
    print(f"{model_name} performance:")
    for metric, value in metrics.items():
        print(f"  {metric}: {value:.4f}")
    print()

In [ ]:
best_model_name_2 = max(results, key=lambda x: results[x]['R^2'])
best_model_regression = models[best_model_name_2]

joblib.dump(best_model_regression, 'best_model_regression.pkl')

# In kết quả của mô hình tốt nhất
print(f"Best Model: {best_model_name_2}")
print(f"Metrics: {results[best_model_name_2]}")

In [ ]:
with open('best_model_regression.pkl', 'wb') as f:
    pickle.dump(best_model_regression, f)

from google.colab import files
files.download('best_mode_regressionl.pkl')

In [ ]:
# Test

# Function to perform frequency encoding
def frequency_encoding(df, column):
    frequency_map = df[column].value_counts(normalize=True)
    df[column + '_freq_encoded'] = df[column].map(frequency_map)
    return df

# New data to predict
instances_to_predict = pd.DataFrame({
    'Delivery Status': ['Advance shipping','Advance shipping'],
    'Category Id': [17,17],
    'Customer City': ['Los Angeles','Los Angeles'],
    'Customer Country': ['EE. UU.', 'EE. UU.'],
    'Customer Segment': ['Corporate','Corporate'],
    'Customer State': ['NY','NY'],
    'Order City': ['Bikaner','Bikaner'],
    'Order Country': ['India','India'],
    'Order Item Product Price': [327.75,327.75],
    'Order Item Quantity': [2,2],
    'Order Status': ['COMPLETE','COMPLETE'],
    'Product Card Id': [1360,1360],
    'Product Price': [327.75,327.75],
    'Order Region': ['Southeast Asia','Southeast Asia'],
    'Market': ['Asia','Asia']
})

# Columns to be frequency encoded
columns_to_encode = ['Delivery Status','Category Name','Customer City','Customer Country','Customer Segment',
                    'Customer State','Order City','Order Country','Order Status','Order Region','Market']

# Perform frequency encoding for each column
for column in columns_to_encode:
    instances_to_predict = frequency_encoding(instances_to_predict, column)

# Drop the original categorical columns
instances_to_predict.drop(columns=columns_to_encode, inplace=True)

# Standardize the features
scaler = StandardScaler()
instances_to_predict_scaled = scaler.fit_transform(instances_to_predict)

# Simulated training data and labels
# Replace these with your actual training data and labels
X_train = np.random.rand(10, instances_to_predict_scaled.shape[1])
y_train = np.random.randint(0, 2, 10)  # Random binary labels

# Train the model
gb_model = make_pipeline(PolynomialFeatures(degree=2), LinearRegression())
gb_model.fit(X_train, y_train)

# Predict whether delivery is late or not
predictions = gb_model.predict(instances_to_predict_scaled)

# Print the predictions
print("Case 1: Predicted Sales:", predictions)
print("Case 2: Predicted Sales:", predictions)